# 001 - Database Connection and Raw Data Loading

This notebook contains the code and the steps necessary to stablish the connection to a PostgreSQL database and load the raw data to be used in posterior analysis. 

The libraries necessary for the correct operation of the code are imported.

In [1]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

### Setting the environment for the connection

In [2]:
try:
    os.chdir("../../Workshop_001")
except FileNotFoundError:
    print("""
        FileNotFoundError - The directory may not exist or you are not located in the specified path.
        """)
os.chdir("..")
print(os.getcwd())

c:\Users\natym\Desktop


### Reading credentials and creating the connection

In [3]:

with open("Workshop_001/credentials.json", "r", encoding = "utf-8") as f:
    credentials = json.load(f)

db_host = credentials["db_host"]
db_name = credentials["db_name"]
db_user = credentials["db_user"]
db_password = credentials["db_password"]

default_engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/postgres")

The database is created

In [4]:
try:
    with default_engine.connect() as connection:
        result = connection.execute(text("SELECT 1 FROM pg_database WHERE datname=:dbname"), {"dbname": db_name})
        exists = result.fetchone()
        if not exists:
            connection.execute(f"CREATE DATABASE {db_name}")
            print("Database created")
        else:
            print("Database already exists")
except OperationalError as e:
    print("PostgreSQL connection failed: {e}")

Database already exists


And at last, the raw data is loaded into the database

In [5]:
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}")

In [ ]:
candidates_raw_data = pd.read_csv("Workshop_001/data/candidates.csv", sep=';')

try:
    candidates_raw_data.to_sql('candidates_raw_data', engine, if_exists='replace', index=False)
    print("Datos insertados correctamente en la tabla 'candidates_raw_data'.")
except Exception as e:
    print(f"Error al insertar datos: {e}")
